**Experimento con LeNet sobre MNIST (Optimización)**

**Lectura del corpus mnist, partición train y test y normalización**

In [1]:
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step
training set (60000, 28, 28)
test set (10000, 28, 28)



**Optimización con conjunto de validación**


Hacemos la partición entrenamiento/validación (80%/20%)

In [2]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (48000, 28, 28)
val set (12000, 28, 28)


**Optimización del modelo en función del learning rate**

In [4]:
from keras.optimizers import SGD

batch_size=128
epochs=5

LR=[0.0001,0.001,0.01,0.1]
best_acc=0.0
for lr in LR:
  M = keras.Sequential()
  M.add(keras.layers.Conv2D(filters=6, kernel_size=(5,5), activation='tanh', input_shape=(28,28,1)))
  M.add(keras.layers.AveragePooling2D(pool_size=(2,2), strides=2))
  M.add(keras.layers.Conv2D(filters=16, kernel_size=(5,5), activation='tanh'))
  M.add(keras.layers.AveragePooling2D(pool_size=(2,2), strides=2))
  M.add(keras.layers.Flatten())
  M.add(keras.layers.Dense(units=120, activation='tanh'))
  M.add(keras.layers.Dense(units=84, activation='tanh'))
  M.add(keras.layers.Dense(units=10, activation = 'softmax'))
  sgd=SGD(learning_rate=lr)
  M.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  print("Learning rate:",lr)
  H = M.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))

  if H.history['val_accuracy'][-1]>best_acc:
        best_acc=H.history['val_accuracy'][-1]
        bestlr=lr

print("Best acc",best_acc)
print("Best learning rate",bestlr)

Learning rate: 0.0001
Epoch 1/5
375/375 [==============================] - 3s 6ms/step - loss: 2.3314 - accuracy: 0.0417 - val_loss: 2.3214 - val_accuracy: 0.0485
Epoch 2/5
375/375 [==============================] - 2s 6ms/step - loss: 2.3143 - accuracy: 0.0566 - val_loss: 2.3049 - val_accuracy: 0.0650
Epoch 3/5
375/375 [==============================] - 2s 5ms/step - loss: 2.2978 - accuracy: 0.0745 - val_loss: 2.2889 - val_accuracy: 0.0857
Epoch 4/5
375/375 [==============================] - 2s 5ms/step - loss: 2.2818 - accuracy: 0.0936 - val_loss: 2.2733 - val_accuracy: 0.1058
Epoch 5/5
375/375 [==============================] - 2s 5ms/step - loss: 2.2661 - accuracy: 0.1129 - val_loss: 2.2579 - val_accuracy: 0.1249
Learning rate: 0.001
Epoch 1/5
375/375 [==============================] - 3s 5ms/step - loss: 2.2412 - accuracy: 0.2404 - val_loss: 2.1445 - val_accuracy: 0.3691
Epoch 2/5
375/375 [==============================] - 2s 7ms/step - loss: 2.0306 - accuracy: 0.5140 - val_loss: 


EARLY STOPPING

El número de épocas a emplear se puede ajustar en función de cómo evoluciona la convergencia sobre el conjunto de validación.


In [5]:
from keras.optimizers import SGD

batch_size=128
epochs=100

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=2)

LR=[0.1]
best_acc=0.0
for lr in LR:
  M = keras.Sequential()
  M.add(keras.layers.Conv2D(filters=6, kernel_size=(5,5), activation='tanh', input_shape=(28,28,1)))
  M.add(keras.layers.AveragePooling2D(pool_size=(2,2), strides=2))
  M.add(keras.layers.Conv2D(filters=16, kernel_size=(5,5), activation='tanh'))
  M.add(keras.layers.AveragePooling2D(pool_size=(2,2), strides=2))
  M.add(keras.layers.Flatten())
  M.add(keras.layers.Dense(units=120, activation='tanh'))
  M.add(keras.layers.Dense(units=84, activation='tanh'))
  M.add(keras.layers.Dense(units=10, activation = 'softmax'))
  sgd=SGD(learning_rate=lr)
  M.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  print("Learning rate:",lr)
  H = M.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),callbacks=[callback])

  if H.history['val_accuracy'][-1]>best_acc:
        best_acc=H.history['val_accuracy'][-1]
        bestlr=lr

print("Best acc",best_acc)
print("Best learning rate",bestlr)

Learning rate: 0.1
Epoch 1/100
375/375 [==============================] - 3s 5ms/step - loss: 0.4624 - accuracy: 0.8715 - val_loss: 0.2350 - val_accuracy: 0.9306
Epoch 2/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1950 - accuracy: 0.9423 - val_loss: 0.1546 - val_accuracy: 0.9540
Epoch 3/100
375/375 [==============================] - 2s 6ms/step - loss: 0.1355 - accuracy: 0.9597 - val_loss: 0.1152 - val_accuracy: 0.9655
Epoch 4/100
375/375 [==============================] - 2s 6ms/step - loss: 0.1050 - accuracy: 0.9682 - val_loss: 0.0967 - val_accuracy: 0.9710
Epoch 5/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0849 - accuracy: 0.9753 - val_loss: 0.0794 - val_accuracy: 0.9761
Epoch 6/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0731 - accuracy: 0.9784 - val_loss: 0.0815 - val_accuracy: 0.9740
Epoch 7/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0636 - accuracy: 0.9804 - val_loss: 0.0673 - v

**Optimización del modelo en función de diferentes optimizadores**

In [6]:
from keras.optimizers import SGD,Adam,Adagrad

batch_size=128
epochs=100
lr=0.1

opt=[]
opt.append(SGD(learning_rate=lr))
opt.append(Adam(learning_rate=lr))
opt.append(Adagrad(learning_rate=lr))

best_acc=0.0
for optim in opt:
    print("Optimizador:",optim)
    M = keras.Sequential()
    M.add(keras.layers.Conv2D(filters=6, kernel_size=(5,5), activation='tanh', input_shape=(28,28,1)))
    M.add(keras.layers.AveragePooling2D(pool_size=(2,2), strides=2))
    M.add(keras.layers.Conv2D(filters=16, kernel_size=(5,5), activation='tanh'))
    M.add(keras.layers.AveragePooling2D(pool_size=(2,2), strides=2))
    M.add(keras.layers.Flatten())
    M.add(keras.layers.Dense(units=120, activation='tanh'))
    M.add(keras.layers.Dense(units=84, activation='tanh'))
    M.add(keras.layers.Dense(units=10, activation = 'softmax'))
    M.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])
    H = M.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),callbacks=[callback])

    if H.history['val_accuracy'][-1]>best_acc:
        best_acc=H.history['val_accuracy'][-1]
        bestopt=optim

print("=============================")
print("Best acc",best_acc)
print("Best optim",bestopt)
print("=============================")

Optimizador: <keras.src.optimizers.sgd.SGD object at 0x7863100ef190>
Epoch 1/100
375/375 [==============================] - 3s 6ms/step - loss: 0.4665 - accuracy: 0.8683 - val_loss: 0.2315 - val_accuracy: 0.9322
Epoch 2/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1876 - accuracy: 0.9443 - val_loss: 0.1449 - val_accuracy: 0.9576
Epoch 3/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1279 - accuracy: 0.9618 - val_loss: 0.1094 - val_accuracy: 0.9684
Epoch 4/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0990 - accuracy: 0.9708 - val_loss: 0.0895 - val_accuracy: 0.9740
Epoch 5/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0816 - accuracy: 0.9760 - val_loss: 0.0773 - val_accuracy: 0.9774
Epoch 6/100
375/375 [==============================] - 2s 6ms/step - loss: 0.0703 - accuracy: 0.9796 - val_loss: 0.0695 - val_accuracy: 0.9794
Epoch 7/100
375/375 [==============================] - 2s 6ms/step - loss